# Project II: Vietnamese Legal Chatbot with Retrieval Augmented Generation(RAG)

Requirement


In [24]:
#%pip install llama_index datasets llama-index-llms-google-genai qdrant_client llama-index-vector-stores-qdrant uuid 

In [25]:
%pip install llama-index-llms-huggingface
%pip install llama-index-llms-huggingface-api
!pip install "transformers[torch]" "huggingface_hub[inference]"


Note: you may need to restart the kernel to use updated packages.


Import

In [26]:
# IGNORE WARNING
import warnings
warnings.filterwarnings("ignore")
from llama_index.core.schema import TextNode
from datasets import load_dataset
import uuid

# LOAD ENV
import os
from dotenv import load_dotenv
load_dotenv()

# LLM
from llama_index.llms.google_genai import GoogleGenAI

# EMBEDDING
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# QDRANT (STORAGE)
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore
#from llama_index.core import StorageContext

In [27]:
import os
from typing import List, Optional

from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

HF_TOKEN: Optional[str] = os.getenv("HUGGING_FACE_TOKEN")


## Setting

In [28]:
!ollama list

NAME              ID              SIZE      MODIFIED    
gemma:7b          a72c7f4d0a15    5.0 GB    4 weeks ago    
gemma2:2b         8ccf136fdd52    1.6 GB    4 weeks ago    
qwen2.5:3b        357c53fb659c    1.9 GB    7 weeks ago    
qwen2.5:1.5b      65ec06548149    986 MB    7 weeks ago    
qwen2.5:latest    845dbda0ea48    4.7 GB    7 weeks ago    


In [29]:
# #%pip install llama-index-llms-ollama
# from llama_index.llms.ollama import Ollama
# llm = Ollama(model="qwen2.5:1.5b", request_timeout=120.0)

In [30]:
# llm.complete("Vì sao bầu trời lại màu xanh")

In [31]:
llm = GoogleGenAI(
    model="gemini-2.0-flash",
)

embed_model = HuggingFaceEmbedding(
    model_name="bkai-foundation-models/vietnamese-bi-encoder"
)

## Data Loading

Danh sách documents chứa các Document, mỗi Document là 1 file về luật hôn nhân gia đình.

Đã load và xử lý index vào qdrant nên không cần load làm gì nữa

In [32]:
# nodes = []
# def _load_dataset(path="khanglt0004/vietnamese_legal_chunks"):
#     dataset = load_dataset(path)
#     for item in dataset['train']:
#         new_node = TextNode(
#             text=item['text'],
#             id_=str(uuid.uuid5(uuid.NAMESPACE_DNS, str(item['id']))),
#             metadata=item['metadata']
#         )
#         nodes.append(new_node)
#     print("Đã tải dữ liệu các chunks, số lượng: ", len(nodes))
# _load_dataset()

## Indexing & Storing

Đã index trong Qdrant Cloud, chỉ việc lấy xuống thôi

In [36]:
import qdrant_client

client = qdrant_client.QdrantClient(
    "https://08838c4e-e0ad-488e-a2a9-b217fa55c19a.us-east-1-0.aws.cloud.qdrant.io",
    api_key=os.environ["QDRANT_API_KEY"]
)


vector_store = QdrantVectorStore(
    client=client, 
    collection_name="law_db",
    enable_hybrid=True,
    fastembed_sparse_model="Qdrant/bm25",
    batch_size=20,
    )


# NOTE: reate Vector Database in Qdrant
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex(
#     nodes,
#     storage_context=storage_context,
#     embed_model = embed_model,
# )

# NOTE: Load Vector Database from Qdrant
loaded_index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model
)

## Retrival 

Dùng để Evaluate khả năng Retrieve của mô hình (@1, @3)

In [37]:
retriever = loaded_index.as_retriever(similarity_top_k=3, sparse_top_k=8, vector_store_query_mode="hybrid")
nodes = retriever.retrieve("Nhà của tôi và chồng đứng tên sổ đỏ, sau khi ly hôn tôi có được quyền sở hữu không?")

In [38]:
for node in nodes:
    print('Score: ', node.score)
    print(node.get_content())
    print('-'*100)

Score:  0.5
Trong trường hợp chỉ một bên có nhu cầu và có điều kiện trực tiếp sử dụng đất thì bên đó được tiếp tục sử dụng nhưng phải thanh toán cho bên kia phần giá trị quyền sử dụng đất mà họ được hưởng;

b) Trong trường hợp vợ chồng có quyền sử dụng đất nông nghiệp trồng cây hàng năm, nuôi trồng thủy sản chung với hộ gia đình thì khi ly hôn phần quyền sử dụng đất của vợ chồng được tách ra và chia theo quy định tại điểm a khoản này;

c) Đối với đất nông nghiệp trồng cây lâu năm, đất lâm nghiệp để trồng rừng, đất ở thì được chia theo quy định tại Điều 59 của Luật này;

d) Đối với loại đất khác thì được chia theo quy định của pháp luật về đất đai.

3. Trong trường hợp vợ chồng sống chung với gia đình mà không có quyền sử dụng đất chung với hộ gia đình thì khi ly hôn quyền lợi của bên không có quyền sử dụng đất và không tiếp tục sống chung với gia đình được giải quyết theo quy định tại Điều 61 của Luật này.
--------------------------------------------------------------------------------

## Query Engine

In [ ]:
# query_engine = loaded_index.as_query_engine(llm=llm, similarity_top_k=3, sparse_top_k=8, vector_store_query_mode="hybrid")

In [ ]:
# response = query_engine.query("Tôi bị chồng hành hung, tôi muốn ly hôn và được nuôi con. Tôi có thể làm gì?")
# print("Câu trả lời:", response.response)
# print("\nTop các đoạn context được dùng:")

# for node in response.source_nodes:
#     print(f"- Score: {node.score}")
#     print(node.node.get_text())
#     print("-----")


## Update Prompt for Retrieval Strategy

Tạo prompt bằng tiếng việt tăng đáng kể hiệu quả

In [39]:
from llama_index.core.prompts import RichPromptTemplate
qa_prompt_tmpl_str = (
    "Bạn là trợ lý tư vấn pháp luật cho nhiệm vụ hỏi đáp với người dùng.\n"
    "Sử dụng các phần sau của bối cảnh được truy xuất để trả lời câu hỏi.\n"
    "Nếu bạn không biết câu trả lời, đừng cố tạo câu trả lời..\n"
    "Ngữ cảnh cung cấp:\n"
    "---------------------\n"
    "{{ context_str }}\n"
    "Hãy trả lời câu hỏi sau với phong cách của một luật sư.\n"
    "Người dùng hỏi: {{ query_str }}\n"
    "Trả lời: "

)
qa_prompt_tmpl = RichPromptTemplate(qa_prompt_tmpl_str)
query_engine_1 = loaded_index.as_query_engine(text_qa_template=qa_prompt_tmpl, llm=llm, similarity_top_k=3, sparse_top_k=8, vector_store_query_mode="hybrid")




In [40]:
# response = query_engine_1.query("Tôi muốn hỏi về thủ tục ly hôn?")
# print("Câu trả lời:", response.response)
# print("\nTop các đoạn context được dùng:")

# for node in response.source_nodes:
#     print(f"- Score: {node.score}")
#     print(node.node.get_text())
#     print("-----")

## Chat Engine

Nếu Query Engine để Q-A 1 1 thì Chat Engine để phục vụ việc chat với mọi người, chat với lịch sử, ..

In [41]:
from llama_index.core import PromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.chat_engine import CondenseQuestionChatEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

sim_postprocessor = SimilarityPostprocessor(similarity_cutoff=0.4)

custom_prompt = PromptTemplate(
    """\
Cho đoạn hội thoại(Giữa người dùng và trợ lý) và một câu hỏi tiếp theo từ người dùng, \
vui lòng viết lại câu hỏi để nó trở thành một câu hỏi độc lập, \
có thể hiểu được toàn bộ ngữ cảnh đoạn hội thoại. \

<Đoạn hội thoại>
{chat_history}

<Câu hỏi tiếp theo>
{question}

<Câu hỏi độc lập>
"""
)

# list of `ChatMessage` objects
custom_chat_history = [
    ChatMessage(
        role=MessageRole.USER,
        content="Chào bạn, tôi cần sự giúp đỡ của bạn về một vấn đề pháp lý, lĩnh vực hôn nhân gia đình.",
    ),
    ChatMessage(role=MessageRole.ASSISTANT, content="Được thôi! Tôi có thể giúp gì cho bạn?"),
]

query_engine = loaded_index.as_query_engine(
    text_qa_template=qa_prompt_tmpl, 
    llm=llm,
    similarity_top_k=3, 
    sparse_top_k=8,
    vector_store_query_mode="hybrid",
    node_postprocessors=[sim_postprocessor],
    )

chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    condense_question_prompt=custom_prompt,
    chat_history=custom_chat_history,
    verbose=True,
    llm=llm,
)



In [42]:
chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    condense_question_prompt=custom_prompt,
    chat_history=custom_chat_history,
    verbose=True,
    llm=llm,
)


In [43]:
chat_engine.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Chào bạn, tôi cần sự giúp đỡ của bạn về một vấn đề pháp lý, lĩnh vực hôn nhân gia đình.')]),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Được thôi! Tôi có thể giúp gì cho bạn?')])]

In [44]:
response = chat_engine.chat("Tôi và chồng không còn tình cảm, chúng tôi đồng thuận ly hôn, chỉ tôi các bước chuẩn bị để ly hôn")
response.response

Querying with: Tôi và chồng không còn tình cảm và cả hai đều đồng ý ly hôn. Xin hãy hướng dẫn các bước cần chuẩn bị để tiến hành thủ tục ly hôn thuận tình.



'Chào bạn,\n\nVới mong muốn ly hôn thuận tình của bạn và chồng, tôi xin cung cấp một số thông tin và hướng dẫn để bạn chuẩn bị cho thủ tục này, dựa trên quy định của pháp luật hiện hành, cụ thể là Điều 55 của Luật Hôn nhân và Gia đình và Nghị quyết 01/2024/NQ-HĐTP hướng dẫn áp dụng một số quy định của pháp luật trong giải quyết vụ việc về hôn nhân và gia đình:\n\n**1. Điều kiện để Tòa án công nhận thuận tình ly hôn:**\n\n*   **Sự tự nguyện:** Cả hai vợ chồng đều thực sự tự nguyện ly hôn. Điều này thể hiện qua việc cả hai cùng ký vào đơn yêu cầu công nhận thuận tình ly hôn, thỏa thuận về việc nuôi con, chia tài sản hoặc một bên có đơn khởi kiện ly hôn, bên kia đồng ý ly hôn và các thỏa thuận liên quan.\n*   **Thỏa thuận về con cái:** Đã có thỏa thuận về việc trông nom, nuôi dưỡng, chăm sóc, giáo dục con chưa thành niên hoặc con đã thành niên mất năng lực hành vi dân sự/không có khả năng lao động và không có tài sản để tự nuôi mình. Thỏa thuận này phải đảm bảo quyền lợi chính đáng của co

In [45]:
response.response

'Chào bạn,\n\nVới mong muốn ly hôn thuận tình của bạn và chồng, tôi xin cung cấp một số thông tin và hướng dẫn để bạn chuẩn bị cho thủ tục này, dựa trên quy định của pháp luật hiện hành, cụ thể là Điều 55 của Luật Hôn nhân và Gia đình và Nghị quyết 01/2024/NQ-HĐTP hướng dẫn áp dụng một số quy định của pháp luật trong giải quyết vụ việc về hôn nhân và gia đình:\n\n**1. Điều kiện để Tòa án công nhận thuận tình ly hôn:**\n\n*   **Sự tự nguyện:** Cả hai vợ chồng đều thực sự tự nguyện ly hôn. Điều này thể hiện qua việc cả hai cùng ký vào đơn yêu cầu công nhận thuận tình ly hôn, thỏa thuận về việc nuôi con, chia tài sản hoặc một bên có đơn khởi kiện ly hôn, bên kia đồng ý ly hôn và các thỏa thuận liên quan.\n*   **Thỏa thuận về con cái:** Đã có thỏa thuận về việc trông nom, nuôi dưỡng, chăm sóc, giáo dục con chưa thành niên hoặc con đã thành niên mất năng lực hành vi dân sự/không có khả năng lao động và không có tài sản để tự nuôi mình. Thỏa thuận này phải đảm bảo quyền lợi chính đáng của co

In [46]:
response_stream = chat_engine.stream_chat("Vậy bây giờ tôi nên làm gì đầu tiên")
response_stream.print_response_stream()

Querying with: Trong trường hợp tôi và chồng đã đồng thuận ly hôn và đã được tư vấn về các bước chuẩn bị hồ sơ, vậy bước đầu tiên tôi nên thực hiện là gì để tiến hành thủ tục ly hôn thuận tình?

Chào bạn,

Dựa trên thông tin bạn cung cấp, tôi hiểu rằng bạn và chồng đã đồng thuận ly hôn và muốn biết bước đầu tiên cần thực hiện để tiến hành thủ tục ly hôn thuận tình.

Tuy nhiên, rất tiếc là với thông tin pháp luật hiện có, tôi chưa thể cung cấp thông tin cụ thể về các bước chuẩn bị hồ sơ ly hôn thuận tình. Để được tư vấn chi tiết và chính xác nhất, bạn nên liên hệ trực tiếp với luật sư hoặc văn phòng luật sư chuyên về lĩnh vực hôn nhân và gia đình. Họ sẽ giúp bạn chuẩn bị đầy đủ hồ sơ và thực hiện các thủ tục pháp lý cần thiết.


In [48]:
# # NOTE: Truy câp lịch sử hội thoại
# chat_engine.chat_history
# # Reset lich sử hội thoại

# # Tạo REPL cho chatbot
# chat_engine.chat_repl()


===== Entering Chat REPL =====
Type "exit" to exit.



In [49]:
chat_engine.chat_history

[]

## Enhance Retrieval 

### Nodes PostProcessor
- Cohere Rerank
- LLM Rerank
- SimilarityPostprocessor

Cohere - Tệ v

In [ ]:
# %pip install llama-index-postprocessor-cohere-rerank 

In [ ]:
# from llama_index.core.postprocessor import SimilarityPostprocessor
# from llama_index.postprocessor.cohere_rerank import CohereRerank
# load_dotenv()

# api_key = os.environ["COHERE_API_KEY"]
# cohere_rerank = CohereRerank(api_key=api_key, top_n=2)

# query_engine_with_coh = loaded_index.as_query_engine(
#     similarity_top_k=6,
#     sparse_top_k=10,
#     vector_store_query_mode="hybrid",
#     llm=llm,
#     text_qa_template=qa_prompt_tmpl,
#     node_postprocessors=[cohere_rerank],
# )



Similarity Processor - Ổn , để cutoff 0.3 hay 0.4 cũng dc

In [ ]:
# from llama_index.core.postprocessor import SimilarityPostprocessor
# postprocessor = SimilarityPostprocessor(similarity_cutoff=0.51)
# query_engine_with_sim = loaded_index.as_query_engine(
#     similarity_top_k=2,
#     sparse_top_k=10,
#     vector_store_query_mode="hybrid",
#     llm=llm,
#     text_qa_template=qa_prompt_tmpl,
#     node_postprocessors=[postprocessor],
# )


In [ ]:
# response = query_engine_with_sim.query(
#     "Chồng tôi có hành vi bạo lực gia đình, tôi muốn ly hôn và được nuôi con. Tôi có thể làm gì?",
# )
# from llama_index.core.response.pprint_utils import pprint_response
# pprint_response(response, show_source=True)